In [ ]:
import  dlc_bci as bci
import torch
from torch import optim
from torch import Tensor
from torch.autograd import Variable
from torch import nn
import numpy as np
from torch.nn import functional as F
from sklearn.linear_model import LogisticRegression

In [ ]:
torch.manual_seed(2)

In [ ]:
train_input, train_target = bci.load(root =  './data')
print(str(type(train_input)), train_input.size())
print(str(type(train_target)), train_target.size())

test_input , test_target = bci.load(root =  './data', train = False)
print(str(type(test_input)), test_input.size())
print(str(type(test_target)), test_target.size())

In [ ]:
# We start by normalize our input datasets
mean, std = train_input.mean(), train_input.std()
train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)

train_input, train_target = Variable(train_input), Variable(train_target)
test_input, test_target = Variable(test_input), Variable(test_target)

train_input_c = train_input.view(train_input.size(0),1,28,-1)
test_input_c = test_input.view(test_input.size(0),1,28,-1)

train_input_f = train_input.view(train_input.size(0),-1)
test_input_f = test_input.view(test_input.size(0),-1)

In [ ]:
def train_model(model, train_input, train_target, mini_batch_size, nb_epochs, eta, sgd=True):
    """
    train our model and print the loss for each chosen epoch
    according to the parameters we pass in the methods arguments
    """
    # We put the model in train mode as we use dropout in some models
    model.train()   
    # We use the cross entropy loss as it give us a better score than MSE
    criterion = nn.CrossEntropyLoss()
    # change the optimizer as we have tried 2 differents
    if (sgd) :
        optimizer = optim.SGD(model.parameters(), lr = eta)
    else :
        optimizer = optim.Adam(model.parameters(), lr=eta)
    # make the forward, backward and optimizer step in order to train our model    
    for e in range(0, nb_epochs):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            optimizer.step()
            sum_loss += loss.data[0]
        print("epoch {} : {}".format(e,sum_loss))

def compute_nb_errors(model, data_input, data_target, mini_batch_size):
    """
    compute our number of errors for our testing dataset with the pretrai model
    """
    # We put the model in evaluation mode in order to avoid the dropout during the test
    model.eval()
    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output.data, 1)
        for k in range(0, mini_batch_size):
            if data_target.data[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

def accuracy(model, train_input, train_target, test_input, test_target, mini_batch_size):
    """
    compute accuracy of training and testing sets.
    """
    acc_train = 100 - compute_nb_errors(model1, train_input, train_target, 4) / train_input.size(0) * 100
    acc_test = 100 - compute_nb_errors(model1, test_input, test_target, 4) / test_input.size(0) * 100
    print("accuracy of the training set : {} %".format(acc_train))
    print("accuracy of the testing set : {} %".format(acc_test))
        

In [ ]:
def fully_connected():
    return nn.Sequential(
        nn.Linear(1400, 2000),
        nn.ReLU(),
        nn.Linear(2000, 2500),
        nn.ReLU(),
        nn.Linear(2500, 1)        
    )

In [ ]:
class Net(nn.Module):
    def __init__(self, nb_hidden):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(64*2*5, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        x = F.tanh(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        x = F.tanh(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        #print(x.size())
        
        x = F.tanh(self.fc1(x.view(-1, 64*2*5)))
        x = self.fc2(x)
        return x

In [ ]:
model1 = Net(250)
train_model(model1, train_input, train_target, mini_batch_size=4, nb_epochs=100, eta=1e-2)

In [ ]:
print(100 - compute_nb_errors(model1, train_input, train_target, 4) / train_input.size(0) * 100)
print(100 - compute_nb_errors(model1, test_input, test_target, 4) / test_input.size(0) * 100)

In [ ]:
class Net2(nn.Module):
    def __init__(self, nb_hidden=150):
        super(Net2, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 28, kernel_size=(28,1), stride=1, padding=0),
            nn.Dropout2d(p=8/28),
            nn.BatchNorm2d(28),
            #nn.MaxPool2d(kernel_size=(1,2), stride=2),
            nn.Tanh())
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(7, 14, kernel_size=(1,4), stride=1, padding=0),
            nn.Dropout(p=25*2/(8*25)),
            nn.BatchNorm2d(14),
            nn.MaxPool2d(kernel_size=3, stride=3),
            nn.Tanh())
        
        self.fc1 = nn.Sequential(
            nn.Linear(14*2*7, nb_hidden),
            nn.ReLU())
        
        self.fc2 = nn.Linear(nb_hidden, 2)
        
        
    def forward(self, x):
        #print(x.size())
        out = self.conv1(x)
        #print(out.size())
        out = self.conv2(out.view(-1,7,8,25))
        #print(out.size())
        out = self.fc1(out.view(-1, 14*2*7))
        #print(out.size())
        out = self.fc2(out)
        #print(out.size())
        return out

In [ ]:
model2 = Net2(14*7)
train_model(model2, train_input, train_target, mini_batch_size=4, nb_epochs=600, eta=1e-2)

In [ ]:
print(100 - compute_nb_errors(model2, train_input, train_target, 4) / train_input.size(0) * 100)
print(100 - compute_nb_errors(model2, test_input, test_target, 4) / test_input.size(0) * 100)

In [ ]:
class Net3(nn.Module):
    def __init__(self, nb_hidden=150):
        super(Net3, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(28,1)),
            nn.Dropout2d(p=6/16),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(kernel_size=(1,2), stride=2),
            nn.Tanh())
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=(1,5)),
            nn.Dropout2d(p=12/32),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=(1,2), stride=2),
            nn.Tanh())
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(2, 8, kernel_size=3),
            nn.Dropout(p=3/8),
            nn.BatchNorm2d(8),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Tanh())
        
        self.fc1 = nn.Sequential(
            nn.Linear(8*7*4, nb_hidden),
            nn.ReLU())
        
        self.fc2 = nn.Linear(nb_hidden, 2)
        
        
    def forward(self, x):
        #print(x.size())
        out = self.conv1(x)
        #print(out.size())
        out = self.conv2(out)
        #print(out.size())
        out = self.conv3(out.view(-1,2,16,10))
        #print(out.size())
        out = self.fc1(out.view(-1, 8*7*4))
        #print(out.size())
        out = self.fc2(out)
        #print(out.size())
        return out

In [ ]:
model3 = Net3(8*7*2)
train_model(model3, train_input, train_target, mini_batch_size=4, nb_epochs=400, eta=1e-2)

In [ ]:
print(100 - compute_nb_errors(model3, train_input, train_target, 4) / train_input.size(0) * 100)
print(100 - compute_nb_errors(model3, test_input, test_target, 4) / test_input.size(0) * 100)

In [ ]:
model4 = Net3(8*7*2)
train_model(model4, train_input, train_target, mini_batch_size=4, nb_epochs=400, eta=1e-2)

In [ ]:
print(100 - compute_nb_errors(model4, train_input, train_target, 4) / train_input.size(0) * 100)
print(100 - compute_nb_errors(model4, test_input, test_target, 4) / test_input.size(0) * 100)

In [ ]:
class Net4(nn.Module):
    def __init__(self, nb_hidden=150):
        super(Net4, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(28,1)),
            nn.Dropout2d(p=8/16),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(kernel_size=(1,2), stride=2),
            nn.Tanh())
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=(1,5)),
            nn.Dropout2d(p=16/32),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=(1,2), stride=2),
            nn.Tanh())
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(2, 8, kernel_size=3),
            nn.Dropout(p=5/8),
            nn.BatchNorm2d(8),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Tanh())
        
        self.fc1 = nn.Sequential(
            nn.Linear(8*7*4, nb_hidden),
            nn.ReLU())
        
        self.fc2 = nn.Linear(nb_hidden, 2)
        
        
    def forward(self, x):
        #print(x.size())
        out = self.conv1(x)
        #print(out.size())
        out = self.conv2(out)
        #print(out.size())
        out = self.conv3(out.view(-1,2,16,10))
        #print(out.size())
        out = self.fc1(out.view(-1, 8*7*4))
        #print(out.size())
        out = self.fc2(out)
        #print(out.size())
        return out

In [ ]:
model5 = Net4(8*7*2)
train_model(model5, train_input, train_target, mini_batch_size=4, nb_epochs=1000, eta=1e-3)

In [ ]:
print(100 - compute_nb_errors(model5, train_input, train_target, 4) / train_input.size(0) * 100)
print(100 - compute_nb_errors(model5, test_input, test_target, 4) / test_input.size(0) * 100)

In [ ]:
hidden = [100,150,200,250,300,350,400,450]
epochs = [50,100,150,200,250,300]

for eta in [1e-1, 1e-2, 1e-3]:
    for nb_hidden in hidden:
        model = Net2(nb_hidden)
        for nb_epochs in epochs:
            train_model(model, train_input, train_target, nb_epochs, eta)
            train_acc = 100 - compute_nb_errors(model, train_input, train_target, 4) / train_input.size(0) * 100
            test_accu = 100 - compute_nb_errors(model, test_input, test_target, 4) / test_input.size(0) * 100
            print("step {}, hidden {}, epoch {}, acc train {}%, acc test {}%".format(eta,nb_hidden, nb_epochs, train_acc, test_accu))